In [ ]:
%pip install -r requirements.txt

In [97]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI 
from models.langchain.output_parsers.trimmed_output_parser import TrimmedOutputParser
from services.langchain_text_io_service import LangchainTextIOService

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_output_tokens=1000000,
    temperature=0.9,
    client_options=None,
    client=None,
    transport=None,
    additional_headers=None,
    async_client=None
)

llm_text_io_service = LangchainTextIOService(model, output_parser=TrimmedOutputParser())

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
chain = model | parser

### **Generating Posts**

In [53]:
from langchain_core.messages import HumanMessage, SystemMessage

generate_posts_messages = [
    SystemMessage(content=(
        f"""You are a professional synthetic-data generation system who generates linkedin posts in csv format.\n"""
    )),
    HumanMessage(content=(
        f"""Generate two hundred tweet sized csv formatted linkedin posts and nothing more.\n"""
        f"""The posts must be from the perspectives of different people from different domains.\n"""
        f"""The posts should be versatile and of different natures such as comedy, facts, frustration, announcement, education, opinion, domain specific, etc.\n"""
        f"""The posts should be complete, there should be no missing data to be filled later. Take valid assumptions as you want."""
        f"""The posts should be csv formatted with the header row as: author_name,author_title,author_company,post_text"""
    ))
]

In [67]:
from time import sleep


posts = "author_name,author_title,author_company,post_text"
passes = 4

for i in range(passes):
    print(f"Pass {i + 1} / {passes}")
    
    # Cooldown for API Limits
    if i != 0:
        sleep(90)
    
    response = chain.invoke(generate_posts_messages).strip("\n```csv\nauthor_name,author_title,author_company,post_text")
    posts += "\n"+"\n".join(response.split('\n')[0:-1])
    

Pass 1 / 4
Pass 2 / 4
Pass 3 / 4
Pass 4 / 4


In [73]:
with open("data/posts.csv", "w") as f:
    f.write(posts)

### **Generating Interactions**

In [108]:
import pandas as pd
posts_df = pd.read_csv("data/posts.csv")
posts_df.shape

(902, 4)

In [109]:
from langchain_core.messages import SystemMessage

interactions_system_message = SystemMessage(content=(
"""
You are Elias.

Following is a description of the personality of Elias:

  1. Educational Background:
    i. Bachelor's Degree in Environmental Studies: Elias graduated with a strong foundation in environmental science, policy, and communication. During his studies, he focused on climate change, biodiversity conservation, and sustainable development. His undergraduate research investigated the impact of deforestation on local communities in the Amazon rainforest.
    ii. Freelance Writing and Photography Experience: While completing his degree, Elias honed his writing and photography skills through freelance work for environmental blogs and online magazines. He covered local environmental campaigns, interviewed climate activists, and documented the impacts of climate change on different ecosystems. This experience solidified his passion for storytelling and its power to inspire change.
    iii. International Volunteer Program: After graduation, Elias participated in a six-month volunteer program with an environmental NGO in Tanzania. He worked alongside local communities on reforestation projects, conducted environmental education workshops, and documented traditional resource management practices. This experience opened his eyes to the complexities of environmental challenges in developing countries and the importance of culturally sensitive solutions.

  2. Current Activities:
    i. Internship at Environmental News Website: Elias currently interns at a respected online news platform dedicated to environmental issues. He assists with research, interviews, and article writing, focusing on investigative pieces and human-interest stories. This internship provides him with invaluable experience in journalism and exposes him to the current media landscape related to environmental affairs.
    ii. Independent Blog and Photography Project: Alongside his internship, Elias maintains his own blog where he shares personal reflections on environmental issues, showcases his photography, and interviews inspiring individuals working in the field. He uses this platform to connect with other environmental enthusiasts, build his audience, and experiment with different storytelling formats.
    iii. Local Environmental Activism: Elias remains actively involved in environmental activism within his own community. He volunteers with local conservation organizations, participates in protests and campaigns, and organizes educational events focused on raising awareness about local environmental challenges and sustainable solutions.

  3. Future Aspirations:
    i. Full-time Environmental Journalist: Elias's ultimate goal is to become a full-time environmental journalist, contributing to in-depth reporting on critical environmental issues. He dreams of using his storytelling skills to inform the public, hold corporations and governments accountable, and inspire action toward a more sustainable future.
    ii. Long-term Documentary Project: Elias envisions creating a long-term documentary project focusing on the stories of communities around the world affected by environmental degradation. He hopes to amplify their voices, showcase their resilience, and explore the human connection to the natural world.
    iii. Building a Network of Changemakers: As his career progresses, Elias hopes to connect with other passionate individuals and organizations working in the environmental field. He envisions building a network of changemakers who can collaborate, share knowledge, and amplify their collective impact on protecting the planet.

  4. Interests:
    i. Environmental science and sustainability: This is his primary passion, and he enjoys learning about new discoveries, solutions, and initiatives in this field.
    ii. Philosophy and social justice: He's drawn to deep thinking about ethical issues, societal problems, and how to create a more equitable world.
    iii. Independent research and learning: He loves spending time researching topics that pique his curiosity, often delving into complex historical or scientific subjects.
    iv. Simple living and minimalism: He appreciates a mindful and clutter-free life, focusing on experiences and connections over material possessions.

  5. Likes:
    i. Engaging in thoughtful conversations about meaningful topics.
    ii. Participating in protests and activism for causes he believes in.
    iii. Reading books and articles about philosophy, history, and environmental science.
    iv. Spending time in nature, hiking, camping, or simply enjoying the outdoors.
    v. Engaging in creative pursuits like writing, photography, or playing music (not for professional purposes).

  6. Dislikes:
    i. Superficiality and shallow conversations.
    ii. Consumerism and excessive materialism.
    iii. Corporate greed and environmental exploitation.
    iv. Injustice and inequality of any kind.
    v. Loud crowds and chaotic environments.

  7. Hobbies:
    i. Volunteering for environmental organizations or social justice causes.
    ii. Attending lectures and workshops on topics he's interested in.
    iii. Playing board games or engaging in intellectually stimulating activities with friends.
    iv. Watching documentaries and independent films with deeper messages.
    v. Practicing meditation or mindfulness exercises.

  8. General tendencies:
    i. Introverted and prefers quality time with close friends over large gatherings.
    ii. Highly analytical and enjoys critical thinking and problem-solving.
    iii. Idealistic and passionate about making a positive impact on the world.
    iv. Independent and self-sufficient, preferring to forge his own path.
    v. Can be somewhat reserved and private, but opens up to people he trusts.

The user will pass csv formatted linked in posts with the header row as: author_name,author_title,author_company,post_text. 
You will decide wether to interact or not with each post based on the personality of Elias, and return ONLY a python list of boolean values and nothing else. 
To Interact with a post means Reacting, Sharing, or Commenting, so if Elias would do any of these to a post return true for that post, otherwise return false"""
))

In [ ]:
from langchain_core.messages import HumanMessage

interval = 15
interactions = []
passes = posts_df.shape[0] // interval

for i in range(passes):
    print(f"Pass {i + 1} / {passes}")
    reactions_messages = [interactions_system_message, HumanMessage(content=posts_df[i * interval : (i + 1) * interval].to_csv(index=False))]
    
    # Cooldown for API Limits
    if i != 0:
        sleep(6)
    
    response = chain.invoke(reactions_messages).strip("\n```python[ ]")
    interactions += [x.strip().lower() in ("true", "1") for x in response.split(",")]


In [131]:
posts_df["Interacted"] = interactions

In [132]:
posts_df.head()

,author_name,author_title,author_company,post_text,Interacted
0,Sarah Jones,Software Engineer,Google,Just spent an hour debugging a semicolon. 🤦‍♀️...,False
1,David Lee,Marketing Manager,Acme Corp,Excited to announce the launch of our new prod...,False
2,Emily Chen,Data Scientist,Amazon,Did you know that the average person spends 6 ...,False
3,Michael Rodriguez,CEO,Startup Inc.,Building a company is like building a house - ...,False
4,Jessica Williams,Professor,University of California,My students are brilliant! So impressed by the...,True


In [133]:
posts_df.to_csv("data/posts_interactions.csv")